In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier
from sklearn import metrics

In [59]:
train = pd.read_csv("dataset/train_values.csv")
train_labels = pd.read_csv("dataset/train_labels.csv")
sub = pd.read_csv("dataset/test_values.csv")

In [60]:
# only select numeric columns and remove building id

easy_train = train.select_dtypes(include='int64').drop(["building_id"], axis=1)
easy_sub = sub.select_dtypes(include='int64').drop(["building_id"], axis=1)
train_labels = train_labels.drop(["building_id"], axis=1)

In [61]:
# split into train test 

X_train, X_test, y_train, y_test = train_test_split(easy_train, train_labels, test_size=0.10, random_state=42)

In [70]:
# load into lgb dataset and specify categorical features

train_data = lgb.Dataset(X_train, label=y_train
#                         feature_name=
#                          ['building_id',
#                          'geo_level_1_id',
#                          'geo_level_2_id',
#                          'geo_level_3_id',
#                          'count_floors_pre_eq',
#                          'age',
#                          'area_percentage',
#                          'height_percentage',
#                          'has_superstructure_adobe_mud',
#                          'has_superstructure_mud_mortar_stone',
#                          'has_superstructure_stone_flag',
#                          'has_superstructure_cement_mortar_stone',
#                          'has_superstructure_mud_mortar_brick',
#                          'has_superstructure_cement_mortar_brick',
#                          'has_superstructure_timber',
#                          'has_superstructure_bamboo',
#                          'has_superstructure_rc_non_engineered',
#                          'has_superstructure_rc_engineered',
#                          'has_superstructure_other',
#                          'count_families',
#                          'has_secondary_use',
#                          'has_secondary_use_agriculture',
#                          'has_secondary_use_hotel',
#                          'has_secondary_use_rental',
#                          'has_secondary_use_institution',
#                          'has_secondary_use_school',
#                          'has_secondary_use_industry',
#                          'has_secondary_use_health_post',
#                          'has_secondary_use_gov_office',
#                          'has_secondary_use_use_police',
#                          'has_secondary_use_other']
#                         categorical_feature=
#                         ['land_surface_condition',
#                          'foundation_type',
#                          'roof_type',
#                          'ground_floor_type',
#                          'other_floor_type',
#                          'position',
#                          'plan_configuration',
#                          'legal_ownership_status']
                        )

In [71]:
validation_data = lgb.Dataset(X_test, label=y_test)

In [72]:
# set params

params = {
    "max_depth" : 5,
    "num_leaves" : 31,
}

In [77]:
num_round = 10


# practice = lgb.train(params, train_data, num_round, valid_sets = [validation_data])

lgb = LGBMClassifier(n_estimators=90, 
                      silent=False, 
                      random_state=43
                     )

In [80]:
lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=90, n_jobs=-1, num_leaves=31, objective=None,
               random_state=43, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [82]:
pred = lgb.predict(X_test)

In [91]:
metrics.f1_score(pred, y_test, average="weighted")

0.7141717961069708

In [95]:
# predict with lgb

lgb_prediction = lgb.predict(easy_sub)

# Naive Prediction with LGBM

In [93]:
submission = pd.read_csv("dataset/submission_format.csv")

In [94]:
submission.head()

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1


In [96]:
submission["damage_grade"] = lgb_prediction

In [99]:
submission.to_csv("submissions/naive_lgb_submission.csv", index=False)